# Environnements de développement Python modernes

Une compréhension approfondie du langage de programmation de choix est la partie la plus importante pour être un expert en programmation. Pourtant, il est vraiment difficile de développer un bon logiciel efcacement sans connaître les meilleurs outils et pratiques qui sont communs au sein de la communauté linguistique donnée. Python n'a pas de fonctionnalité unique qui ne peut être trouvée dans un autre langage. Ainsi, lorsque l'on compare la syntaxe, l'expressivité ou les performances, il y aura toujours une solution meilleure dans un ou plusieurs domaines. Mais le domaine dans lequel Python se démarque vraiment de la foule est l'ensemble de l'écosystème construit autour du langage. La communauté Python a passé de nombreuses années à peaufiner les pratiques et les bibliothèques standard qui aident à créer des logiciels de haute qualité en moins de temps.

L'écriture de nouveaux logiciels est toujours un processus long et coûteux. Cependant, pouvoir réutiliser le code existant au lieu de réinventer la roue réduit considérablement les délais et les coûts de développement. Pour certaines entreprises, c'est la seule raison pour laquelle leurs projets sont économiquement réalisables. C'est pourquoi la partie la plus importante de l'écosystème est une énorme collection d'emballages réutilisables qui résolvent une multitude de problèmes. Un grand nombre de ces packages sont disponibles en open source via le Python Package Index (PyPI)

En raison de l'importance de la communauté open source de Python, les développeurs Python ont consacré beaucoup d'efforts à la création d'outils et de normes pour travailler avec des packages Python créés par d'autres, à partir d'environnements virtuels isolés, de shells interactifs améliorés et de débogueurs, aux utilitaires. qui vous aident à découvrir, rechercher et analyser l'énorme collection de packages disponibles sur PyPI

## L'écosystème packaging de Python

Le cœur de l'écosystème packaging de Python est le Python Packaging Index. PyPI est un vaste référentiel public de projets Python (principalement) gratuits qui, au moment de la rédaction, hébergent près de trois millions et demi de distributions de plus de 250 000 packages. Ce n'est pas le plus grand nombre parmi tous les référentiels de packages (npm a dépassé le million de packages en 2019), mais cela place toujours Python parmi les leaders des écosystèmes packaging.

Un si grand écosystème de packages n'est pas gratuit. Les applications modernes sont souvent construites à l'aide de plusieurs packages de PyPI qui ont souvent leurs propres dépendances. Ces dépendances peuvent également avoir leurs propres dépendances. Dans les grandes applications, de telles chaînes de dépendances peuvent continuer indéfiniment. Ajoutez le fait que certains packages peuvent nécessiter des versions spécifiques d'autres packages et vous pouvez rapidement vous retrouver dans l'enfer des dépendances - une situation où il est presque impossible de résoudre manuellement les exigences de version conflictuelles.

## Isoler l'environnement d'exécution

Lorsque vous utilisez pip pour installer un nouveau package à partir de PyPI, il sera installé dans l'un des répertoires de site-packages disponibles. L'emplacement exact des répertoires de site-packages est spécifique au système d'exploitation. Vous pouvez inspecter les chemins où Python recherchera des modules et des packages en utilisant le module de site comme commande comme suit

In [1]:
! python3 -m site

sys.path = [
    '/content',
    '/env/python',
    '/usr/lib/python37.zip',
    '/usr/lib/python3.7',
    '/usr/lib/python3.7/lib-dynload',
    '/usr/local/lib/python3.7/dist-packages',
    '/usr/lib/python3/dist-packages',
]
USER_BASE: '/root/.local' (exists)
USER_SITE: '/root/.local/lib/python3.7/site-packages' (doesn't exist)
ENABLE_USER_SITE: True


La variable sys.path dans la sortie précédente est une liste d'emplacements de recherche de module. Ce sont des emplacements à partir desquels Python tentera de charger les modules. La première entrée est toujours le répertoire de travail actuel (dans ce cas, /content) et la dernière est le répertoire global site-packages, souvent appelé répertoire dist-packages.

Le USER_SITE dans la sortie précédente décrit l'emplacement du répertoire utilisateur site-packages, qui est toujours spécifique à l'utilisateur qui appelle actuellement l'interpréteur Python. Les packages installés dans un répertoire site-packages local auront priorité sur les packages installés dans le répertoire global site-packages.

Une autre façon d'obtenir les packages de site consiste à invoquer sys.getsitepackages(). Voici un exemple d'utilisation de cette fonction dans un shell interactif :

In [2]:
import site
site.getsitepackages()

['/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.7/dist-packages']

Vous pouvez également obtenir les répertoires de packages de sites utilisateur en appelant la fonction sys.getusersitepackages() comme suit :

In [4]:
import site

site.getusersitepackages()

'/root/.local/lib/python3.7/site-packages'

Lors de l'exécution de pip install, les packages seront installés dans le répertoire utilisateur ou global site-packages en fonction de plusieurs conditions évaluées dans l'ordre suivant:

* **1** **user site-packages** si le paramètre --user est spécifié
* **2** **global site-packages** si le répertoire global site-packages est accessible en écriture à l'utilisateur appelant pip
* **3** **user site-packages** autre cas 

Les conditions précédentes signifient simplement que sans le commut
ateur --user, pip tentera toujours d'installer des packages dans un répertoire global site-packages et ne reviendra aux sites-packages utilisateur que si cela n'est pas possible. Sur la plupart des systèmes d'exploitation où Python est disponible par défaut (nombreuses distributions Linux, macOS), le répertoire global site-packages de la distribution Python du système est protégé contre les écritures d'utilisateurs non privilégiés. Cela signifie que pour installer un package dans le répertoire global site-packages à l'aide des distributions Python d'un système, vous devrez utiliser une commande qui vous accorde des privilèges de superutilisateur, comme sudo. Sur les systèmes de type UNIX et Linux, l'invocation de pip par le superutilisateur sera la suivante:

    sudo -H pip install <package-name>

    

    Les privilèges de superutilisateur pour l'installation de packages Python à 
    al'échelle du système ne sont pas requis sous Windows car il ne fournit pas 
    l'interpréteur Python par défaut. De plus, pour certains autres systèmes 
    d'exploitation (comme macOS), si vous installez Python à partir du 
    programme d'installation disponible sur le site Web python.org, il sera 
    installé de manière à ce que le répertoire global site-packages soit 
    accessible en écriture aux utilisateurs normaux.

Bien que l'installation de packages directement depuis PyPI dans le répertoire global site-packages soit possible et dans certains environnements se produise par défaut, cela n'est généralement pas recommandé et doit être évité. Gardez à l'esprit que pip n'installera qu'une seule version d'un package dans le répertoire site-packages. Si une ancienne version est déjà disponible, la nouvelle installation l'écrasera. Cela peut être problématique, surtout si vous envisagez de créer différentes applications avec Python. Recommander de ne rien installer dans le répertoire global site-packages peut sembler déroutant car c'est le comportement par défaut de pip, mais il y a de sérieuses raisons à cela.

Comme nous l'avons mentionné précédemment, Python est souvent une partie importante de nombreux packages disponibles via les référentiels de packages du système d'exploitation et peut alimenter de nombreux services importants. Les responsables de la distribution du système mettent beaucoup d'efforts pour sélectionner les versions correctes des packages pour correspondre aux différentes dépendances des packages.

Très souvent, les packages Python disponibles dans les référentiels de packages d'un système (comme apt, yum ou rpm) contiennent des correctifs personnalisés ou sont volontairement tenus à jour pour assurer la compatibilité avec certains autres composants du système. Forcer la mise à jour d'un tel package, à l'aide de pip, vers une version qui rompt une certaine compatibilité descendante peut provoquer des bogues dans certains services système cruciaux.

Enfin et surtout, si vous travaillez sur plusieurs projets en parallèle, vous remarquerez qu'il est pratiquement impossible de maintenir une seule liste de versions de package qui fonctionne pour tous vos projets. Les packages évoluent rapidement et toutes les modifications ne sont pas rétrocompatibles. Vous finirez par vous retrouver dans une situation où l'un de vos nouveaux projets a désespérément besoin de la dernière version d'une bibliothèque, mais qu'un autre projet ne peut pas l'utiliser car il y a un changement incompatible avec les versions antérieures. Si vous installez un package dans des packages de site globaux, vous ne pourrez utiliser qu'une seule version de ce package.


Heureusement, il existe une solution simple à ce problème : l'isolation de l'environnement. Il existe divers outils qui permettent d'isoler l'environnement d'exécution Python à différents niveaux d'abstraction du système. L'idée principale est d'isoler les dépendances du projet des packages requis par différents projets et/ou services système. Les avantages de cette approche sont les suivants

* Il résout le projet X qui dépend du package 1.x mais le projet Y a besoin du dilemme du package 4.x. Le développeur peut travailler sur plusieurs projets avec des dépendances différentes qui peuvent même entrer en collision sans risque de s'affecter. 

*  Le projet n'est plus limité par les versions des packages fournis dans les référentiels de distribution système du développeur (comme apt, yum, rpm, et ainsi de suite).

*  Il n'y a aucun risque de casser d'autres services système qui dépendent de certaines versions de package, car les nouvelles versions de package ne sont disponibles qu'à l'intérieur d'un tel environnement.

* Une liste de packages qui sont des dépendances de projet peut être facilement verrouillée. Le verrouillage capture généralement les versions exactes de tous les packages dans toutes les chaînes de dépendances, il est donc très facile de reproduire un tel environnement sur un autre ordinateur.

Si vous travaillez sur plusieurs projets en parallèle, vous découvrirez rapidement qu'il est impossible de maintenir leurs dépendances sans une sorte d'isolement

Discutons de la différence entre l'isolation au niveau de l'application et l'isolation au niveau du système dans la section suivante


## Isolation au niveau de l'application par rapport à l'isolation au niveau du système

L'approche la plus simple et la plus légère de l'isolation consiste à utiliser l'isolation au niveau de l'application via des environnements virtuels. Python a un module venv intégré qui simplifie grandement l'utilisation et la création de tels environnements virtuels.

Les environnements virtuels se concentrent sur l'isolement de l'interpréteur Python et des packages disponibles à l'intérieur. De tels environnements sont très faciles à configurer mais ne sont pas portables, principalement parce qu'ils reposent sur des chemins système absolus. Cela signifie qu'ils ne peuvent pas être facilement copiés entre les ordinateurs et les systèmes d'exploitation sans casser les choses. Ils ne peuvent même pas être déplacés entre les répertoires du même système de fichiers. Néanmoins, ils sont suffisamment robustes pour assurer une isolation adéquate lors du développement de petits projets et packages. Grâce à la prise en charge intégrée dans les distributions Python, ils peuvent également être facilement répliqués par vos pairs.

Les environnements virtuels sont généralement suffisants pour écrire des bibliothèques ciblées qui sont indépendantes du système d'exploitation ou des projets de faible complexité qui n'ont pas trop de dépendances externes. De plus, si vous écrivez un logiciel qui doit être exécuté uniquement sur votre propre ordinateur, les environnements virtuels devraient suffire à fournir une isolation et une reproductibilité sufsantes.

Malheureusement, dans certains cas, cela peut ne pas être suffisant pour assurer une cohérence et une reproductibilité suffisantes. Malgré le fait que les logiciels écrits en Python soient généralement considérés comme très portables, tous les packages ne se comporteront pas de la même manière sur tous les systèmes d'exploitation. Cela est particulièrement vrai pour les packages qui s'appuient sur des bibliothèques partagées tierces (DLL sous Windows, .so sous Linux, .dylib sous macOS) ou font un usage intensif d'extensions Python compilées écrites en C ou C++, mais cela peut également se produire pour Les bibliothèques Python qui utilisent des API spécifiques à un système d'exploitation donné

Dans de tels cas, l'isolation au niveau du système est un bon ajout au flux de travail. Ce type d'approche essaie généralement de répliquer et d'isoler des systèmes d'exploitation complets avec toutes leurs bibliothèques et composants système cruciaux, soit avec des outils de virtualisation de système d'exploitation classiques (par exemple, VMware, Parallels et VirtualBox) ou des systèmes de conteneurs (par exemple, Docker et Rocket). Certaines des solutions disponibles qui fournissent ce type d'isolation sont détaillées plus loin dans la section Isolation de l'environnement au niveau du système.

L'isolation au niveau du système devrait être votre option préférée pour l'environnement de développement si vous écrivez un logiciel sur un ordinateur différent de celui sur lequel vous l'exécuterez. Si vous exécutez votre logiciel sur des serveurs distants, vous devez absolument envisager l'isolement au niveau du système dès le début, car cela peut vous éviter des problèmes de portabilité à l'avenir. Et vous devriez le faire, que votre application repose ou non sur du code compilé (bibliothèques partagées, extensions compilées). L'utilisation de l'isolation au niveau du système vaut également la peine d'être envisagée si votre application fait un usage intensif de services externes tels que des bases de données, des caches, des moteurs de recherche, etc. En effet, de nombreuses solutions d'isolation au niveau du système vous permettent également d'isoler facilement ces dépendances.

Étant donné que les deux approches de l'isolation de l'environnement ont leur place dans le développement Python moderne, nous les aborderons toutes les deux en détail. Commençons par le plus simple : les environnements virtuels utilisant le module venv de Python.

### Isolation de l'environnement au niveau de l'application

Python a un support intégré pour la création d'environnements virtuels. Il se présente sous la forme d'un module venv qui peut être appelé directement depuis votre shell système. Pour créer un nouvel environnement virtuel, utilisez simplement la commande suivante

    python3.9 -m venv <env-name>

Ici, env-name doit être remplacé par le nom souhaité pour le nouvel environnement (il peut également s'agir d'un chemin absolu). Notez comment nous avons utilisé la commande python3.9 au lieu de python3 simple. En effet, selon l'environnement, python3 peut être lié à différentes versions d'interpréteur et il est toujours préférable d'être très explicite sur la version de Python lors de la création de nouveaux environnements virtuels. Les commandes python3.9 -m venv créeront un nouveau répertoire env-name dans le chemin du répertoire de travail actuel. A l'intérieur, il contiendra quelques sous-répertoires

* bin/ : c'est là que le nouvel exécutable Python et les scripts/exécutables fournis par d'autres packages sont stockés

* lib/ et include/ : ces répertoires contiennent les fichiers de bibliothèque de prise en charge du nouvel interpréteur Python dans l'environnement virtuel. De nouveaux packages seront installés dans ENV-NAME/lib/pythonX.Y/site-packages/


Une fois le nouvel environnement créé, il doit être activé dans la session shell en cours. Si vous utilisez Bash comme shell, vous pouvez activer l'environnement virtuel à l'aide de la commande source

    source env-name/bin/activate

Il existe également une version plus courte qui devrait fonctionner sous n'importe quel système compatible POSIX, quel que soit le shell :

    . env-name/bin/activate

Cela modifie l'état de la session shell actuelle en affectant ses variables d'environnement. Afin d'informer l'utilisateur qu'il a activé l'environnement virtuel, il modifiera l'invite du shell en ajoutant la chaîne (ENV-NAME) à son début. Pour illustrer cela, voici un exemple de session qui crée un nouvel environnement et l'active

    $ python3 -m venv example
    $ source example/bin/activate(example) 
    $ which python/home/swistakm/example/bin/python
    (example) $ deactivate
    $ which python
    /usr/local/bin/python

La chose importante à noter à propos de venv est qu'il ne fournit aucune capacité supplémentaire pour suivre les paquets qui doivent y être installés. Les environnements virtuels ne sont pas non plus portables et ne doivent pas être déplacés vers un autre système/machine ou même un chemin de système de fichiers différent. Cela signifie qu'un nouvel environnement virtuel doit être créé à chaque fois que vous souhaitez installer votre application sur un nouvel hôte

Pour cette raison, il existe une meilleure pratique utilisée par les utilisateurs de pip pour stocker la définition de toutes les dépendances du projet en un seul endroit. La façon la plus simple de le faire est de créer un fichier requirements.txt (c'est la convention de nommage), avec le contenu comme indiqué dans le code suivant

    # pinned version specifiers are best for reproducibility
    eventlet==0.17.4
    graceful==0.1.1

    # for projects that are well tested with different 
    # dependency versions the version ranges are acceptable
    falcon>=0.3.0,<0.5.0

    # packages without versions should be avoided unless 
    # latest release is always required/desired pytz

Avec un tel fichier, toutes les dépendances peuvent être facilement installées en une seule étape. La commande pip install comprend le format de ces fichiers requirements . Vous pouvez spécifier le chemin d'accès à un fichier d'exigences à l'aide de l'indicateur -r comme dans l'exemple suivant :

    pip install -r requirements.txt

N'oubliez pas que les fichiers requirements  spécifient uniquement les packages à installer et non les packages qui se trouvent actuellement dans votre environnement. Si vous installez quelque chose manuellement dans votre environnement, cela ne sera pas automatiquement reflété dans votre fichier requirements . Ainsi, un grand soin doit être apporté à la mise à jour de votre fichier requirements , en particulier pour les projets importants et complexes.

Il existe la commande pip freeze, qui imprime tous les packages de l'environnement actuel avec leurs versions, mais elle doit être utilisée avec précaution. Cette liste comprendra également les dépendances de vos dépendances, donc pour les gros projets, elle deviendra rapidement très volumineuse. Vous devrez inspecter soigneusement si la liste contient quelque chose d'installé accidentellement ou par erreur

Pour les projets qui nécessitent une meilleure reproductibilité des environnements virtuels et un contrôle strict des dépendances installées, vous aurez peut-être besoin d'un outil plus sophistiqué. Nous discuterons d'un tel outil - **Poetry** - dans la section suivante

#### Poetry comme système de gestion des dépendances

Poetry est une approche assez nouvelle de la gestion des dépendances et de l'environnement virtuel en Python. Il s'agit d'un projet open source qui vise à fournir un environnement plus prévisible et plus pratique pour travailler avec l'écosystème pakaging Python

    pip install --user poetry

Comme déjà souligné dans la section Installer des packages Python à l'aide de pip, la commande ci-dessus installera le package Poetry dans votre répertoire site-packages. Selon la configuration de votre système, il s'agira soit du répertoire global site-packages, soit du répertoire utilisateur site-packages. Pour éviter cette ambiguïté, les créateurs du projet Poetry recommandent d'utiliser une méthode d'amorçage alternative.

Sur macOS, Linux et autres systèmes compatibles POSIX, Poetry peut être installé à l'aide de l'utilitaire curl

    curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python

Sous Windows, il peut être installé à l'aide de PowerShell

    (Invoke-WebRequest -Uri https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py -UseBasicParsing).Content | python -


Une fois installé, Poetry peut être utilisé pour

* Créer de nouveaux projets Python avec des environnements virtuels
* Initialiser des projets existants avec un environnement virtuel
*  Gérer les dépendances du projet
*  Bibliothèques de packages


Pour créer un tout nouveau projet avec Poetry, vous pouvez utiliser la commande POETRY new comme dans l'exemple suivant

    poetry new my-project

La commande ci-dessus créera un nouveau répertoire my-project contenant quelques fichiers initiaux. La structure de ce répertoire sera à peu près comme suit/

    my-project/
    ├── README.rst
    ├── my_project
    │   └── __init__.py
    ├── pyproject.toml
    └── tests    
        ├── __init__.py    
        └── test_my_project.py
  

Comme vous pouvez le voir, cela crée des FICHIERS qui peuvent être utilisés comme stubs pour un développement ultérieur. Si vous avez un projet préexistant, vous pouvez initialiser Poetry à l'intérieur de celui-ci à l'aide de la commande PoeTRY init à l'intérieur de votre répertoire de projet. La différence est qu'il ne créera aucun nouveau fichier de projet à l'exception du fichier de configuration pyproject.toml

Le noyau de Poetry est le fichier pyproject.toml, qui stocke la configuration du projet. Pour l'exemple mon-projet, il peut avoir le contenu suivant :

    [tool.poetry]
    name = "my-project
    "version = "0.1.0"
    description = ""
    authors = [" "]

    [tool.poetry.dependencies]
    python = "^3.9"
    
    [tool.poetry.dev-dependencies]
    pytest = "^5.2"
    
    [build-system]
    requires = ["poetry-core>=1.0.0"]
    build-backend = "poetry.core.masonry.api"


Comme vous pouvez le voir, le fichier pyproject.toml est divisé en quatre sections. Ce sont :

*  [tool.poetry] : il s'agit d'un ensemble de métadonnées de projet de base telles que le nom, la description de la version et l'auteur. Ces informations sont nécessaires si vous souhaitez publier votre projet sous forme de package sur PyPI.

* [tool.poetry.dependencies] : il s'agit d'une liste des dépendances du projet. Sur les nouveaux projets, il répertorie uniquement la version Python mais peut également inclure toutes les versions de package qui seraient normalement décrites dans le fichier requirements.txt.

*  [tool.poetry.dev-dependencies] : il s'agit d'une liste de dépendances qui nécessitent des développement, comme des frameworks de test ou des outils de productivité. Il est courant d'avoir une liste séparée de ces dépendances, car elles ne sont généralement pas nécessaires dans les environnements de production.

* [build-system] : décrit Poetry comme un système de build utilisé pour gérer le projet


Si vous créez un nouveau projet ou initialisez un existant à l'aide de Poetry, il pourra créer un nouvel environnement virtuel dans un emplacement partagé chaque fois que vous en aurez besoin. Vous pouvez l'activer en utilisant Poetry au lieu de "sourcer" les scripts d'activation. C'est plus pratique que d'utiliser le module venv simple car vous n'avez pas besoin de vous rappeler où l'environnement virtuel réel est stocké. La seule chose que vous devez faire est de déplacer votre shell n'importe où dans l'arborescence source de votre projet et d'utiliser la commande de shell poetry comme dans l'exemple suivant

    cd my-project
    poetry shell

A partir de ce moment, le shell actuel aura l'environnement virtuel de Poetry activé. Vous pouvez le vérifier avec la commande which python ou python -m site

Une autre chose que poetry change est la façon dont vous gérez les dépendances. Comme nous l'avons déjà mentionné, les fichier requirements.txt sont un moyen très basique de gérer les dépendances. Ils décrivent les packages à installer mais ne suivent pas automatiquement ce qui a été installé dans l'environnement tout au long du développement. Si vous installez quelque chose avec pip mais oubliez de refléter ce changement dans le fichier requirements.txt, d'autres programmeurs peuvent avoir des problèmes pour recréer votre environnement.


Avec poetry, ce problème a disparu. Il n'y a qu'une seule façon d'ajouter des dépendances à votre projet et c'est avec la commande poetry add <package-name>. Ce sera

* Résoudre des arborescences de dépendances entières si d'autres packages partagent des dépendances

* Installez tous les packages de l'arborescence de dépendances dans l'environnement virtuel associé à votre projet

* Reflètez le changement dans le fichier pyproject.toml

La transcription suivante présente le processus d'installation du framework Flask dans l'environnement my-project :

    poetry add flask


Et ce qui suit est le fichier pyproject.toml résultant avec les modifications mises en évidence dans les dépendances du projet

    [tool.poetry]
    name = "my-project"version = "0.1.0"
    description = ""
    authors = [""]
    
    [tool.poetry.dependencies]
    python = "^3.9"
    Flask = "^1.1.2"
    
    [tool.poetry.dev-dependencies]
    pytest = "^5.2"
    
    [build-system]
    requires = ["poetry-core>=1.0.0

La transcription précédente montre que Poetry a installé 15 packages alors que nous n'avons demandé qu'une seule dépendance. C'est parce que Flask a ses propres dépendances et ces dépendances ont leurs propres dépendances. De telles dépendances de dépendances sont appelées dépendances transitives. Les bibliothèques ont souvent des spécificateurs de version laxistes comme six >=1.0.0 pour indiquer qu'elles sont capables d'accepter un large éventail de versions. Poetry implémente un algorithme de résolution de dépendance pour déterminer quel ensemble de versions peut satisfaire toutes les contraintes de dépendance transitive de dépendance

Le problème avec les dépendances transitives est leur capacité à changer au fil du temps. N'oubliez pas que les bibliothèques peuvent avoir des spécificateurs de version laxistes pour leurs dépendances. Il est donc possible que deux environnements créés à des dates différentes aient des versions nales différentes des packages installés. L'incapacité à reproduire les versions exactes de toutes les dépendances transitives peut être un gros problème pour les grands projets et leur suivi manuel dans les fichiers requirements.txt est généralement un gros défi

Poetry résout le problème des dépendances transitives en utilisant des fichiers de verrouillage de dépendance. Chaque fois que vous êtes sûr que votre environnement dispose d'un ensemble fonctionnel et testé de versions de package, vous pouvez exécuter la commande suivante:

    poetry lock

Cela créera un fichier poetry.lock très détaillé qui est un instantané complet du processus de résolution des dépendances. Ce fichier sera ensuite utilisé pour déterminer les versions des dépendances transitives au lieu du processus de dépendance ordinaire.

Chaque fois que de nouveaux packages sont ajoutés avec la commande Poetry add, Poetry évaluera l'arbre de dépendances et mettra à jour le fichier Poesie.lock. L'approche de fichier de verrouillage est jusqu'à présent le moyen le meilleur et le plus fiable de gérer les dépendances transitives dans votre projet

## Isolation de l'environnement au niveau du système

Le facteur clé de l'itération rapide de la mise en œuvre du logiciel est la réutilisation des composants logiciels existants. Ne vous répétez pas, c'est un mantra commun à de nombreux programmeurs. L'utilisation d'autres packages et modules pour les inclure dans la base de code n'est qu'une partie de cet état d'esprit. Ce qui peut également être considéré comme des composants réutilisés sont les bibliothèques binaires, les bases de données, les services système, les API tierces, etc. Même les systèmes d'exploitation entiers doivent être considérés comme un composant réutilisé.

Les services principaux des applications Web sont un excellent exemple de la complexité de ces applications. La pile logicielle la plus simple se compose généralement de quelques couches. Considérez une application imaginaire qui vous permet de stocker des informations sur ses utilisateurs et de les exposer à Internet via le protocole HTTP. Il pourrait avoir au moins les trois couches suivantes (en commençant par la plus basse) :

* Une base de données ou un autre type de moteur de stockage
* Le code d'application implémenté en Python
* Un serveur HTTP fonctionnant en mode proxy inverse, tel qu'Apache ou NGINX

Bien que des applications très simples puissent être à une seule couche, cela arrive rarement pour des applications complexes ou des applications conçues pour gérer un trafic très important. En fait, les grosses applications sont parfois si complexes qu'elles ne peuvent pas être représentées comme un empilement de couches mais plutôt comme un patchwork ou un maillage de services interconnectés. Les petites et les grandes applications peuvent utiliser de nombreuses bases de données différentes, être divisées en plusieurs processus indépendants et utiliser de nombreux autres services système pour la mise en cache, la mise en file d'attente, la journalisation, la découverte de services, etc. Malheureusement, il n'y a pas de limites à cette complexité.

Ce qui est vraiment important, c'est que tous les éléments de la pile logicielle ne peuvent pas être isolés au niveau des environnements d'exécution Python. Qu'il s'agisse d'un serveur HTTP, tel que NGINX, ou d'un SGBDR, tel que PostgreSQL, ou d'une bibliothèque partagée, ces éléments ne font généralement pas partie de la distribution Python ou de l'écosystème de packages Python et ne peuvent pas être encapsulés dans les environnements virtuels de Python. . C'est pourquoi ils sont considérés comme des dépendances externes de votre logiciel.

Ce qui est très important, c'est que les dépendances externes sont généralement disponibles dans différentes versions  sur différents systèmes d'exploitation. Par exemple, si deux développeurs utilisent des distributions Linux complètement différentes, disons Debian et Gentoo, il est vraiment peu probable qu'ils aient accès à un moment donné à la même version de logiciel comme NGINX via les référentiels de packages de leur système. De plus, ils peuvent être compilés à l'aide de différents indicateurs de compilation (par exemple, en activant des paramètres spécifiques), ou être fournis avec des extensions personnalisées ou des correctifs spécifiques à la distribution.

Ainsi, s'assurer que tout le monde dans une équipe de développement utilise les mêmes versions de chaque composant est très difficile sans les outils appropriés. Il est théoriquement possible que tous les développeurs d'une équipe travaillant sur un même projet puissent obtenir les mêmes versions de services sur leurs box de développement. Mais tous ces efforts sont vains s'ils n'utilisent pas le même système d'exploitation que dans leur environnement de production. Forcer un programmeur à travailler sur autre chose plutôt que sur son système de choix bien-aimé n'est pas non plus toujours possible

    L'environnement de production, ou production en abrégé, est l'environnement
    réel dans lequel votre application est installée et s'exécute pour remplir
    son rôle. Par exemple, l'environnement de production d'une application de
    bureau serait l'ordinateur de bureau réel sur lequel vos utilisateurs
    installent leurs applications. L'environnement de production d'un serveur
    principal pour une application Web disponible via Internet est généralement
    un serveur distant (parfois virtuel) fonctionnant dans une sorte de centre
    de données


Le problème réside dans le fait que la portabilité est toujours un grand défi. Tous les services ne fonctionneront pas exactement de la même manière dans les environnements de production que sur les machines des développeurs. Et il est peu probable que cela change. Même Python peut se comporter différemment sur différents systèmes, malgré le travail accompli pour le rendre multiplateforme. Habituellement, pour Python, cela est bien documenté et ne se produit que dans des endroits qui interagissent directement avec le système d'exploitation. Pourtant, s'appuyer sur la capacité du programmeur à se souvenir d'une longue liste de problèmes de compatibilité est une stratégie assez sujette aux erreurs.


Une solution courante à ce problème consiste à isoler des systèmes entiers en tant qu'environnement d'application. Ceci est généralement réalisé en tirant parti de différents types d'outils de virtualisation du système. La virtualisation, bien sûr, peut avoir un impact sur les performances ; mais avec les processeurs modernes qui prennent en charge matériellement la virtualisation, la perte de performances est considérablement réduite. Par contre, la liste des gains possibles est très longue.

* L'environnement de développement peut correspondre exactement à la version du système, aux services et aux bibliothèques partagées utilisés en production, ce qui permet de résoudre les problèmes de compatibilité.

* Dénitions des outils de configuration du système, tels que Puppet, Chef ou Ansible (le cas échéant), peut être réutilisé pour configurer à la fois les environnements de production et de développement.

* Les membres de l'équipe nouvellement embauchés peuvent facilement se joindre au projet si la création de tels environnements est automatisée.

* Les développeurs peuvent travailler directement avec des fonctionnalités système de bas niveau qui peuvent ne pas être disponibles sur les systèmes d'exploitation qu'ils utilisent pour le travail. Par exemple, le système de fichiers dans l'espace utilisateur (FUSE) est une fonctionnalité des systèmes d'exploitation Linux avec laquelle vous ne pouvez pas travailler sous Windows sans virtualisation.

Dans la section suivante, nous examinerons deux approches différentes pour parvenir à l'isolation au niveau du système des environnements de développement

## Conteneurisation versus virtualisation

Les techniques d'isolation au niveau du système peuvent être utilisées de deux manières principales à des fins de développement:

* ** Virtualisation des machines**, qui émule l'ensemble du système informatique
*  **Virtualisation au niveau du système d'exploitation**, également appelée conteneurisation, qui isole des espaces utilisateur complets au sein d'un seul système d'exploitation


Les techniques de virtualisation des machines se concentrent sur l'émulation de systèmes informatiques complets au sein d'autres systèmes informatiques. Considérez-le comme un matériel virtuel pouvant être exécuté en tant que logiciel sur votre propre ordinateur. Comme il s'agit d'une émulation matérielle complète, cela vous donne la possibilité d'exécuter n'importe quel système d'exploitation dans vos environnements hôtes. Il s'agit de la technologie qui pilote l'infrastructure des serveurs privés virtuels (VPS) et des fournisseurs de cloud computing, car elle vous permet d'exécuter plusieurs systèmes d'exploitation indépendants et isolés au sein d'un seul ordinateur hôte.

C'est également une méthode pratique pour exécuter de nombreux systèmes d'exploitation à des fins de développement, car le démarrage d'un nouveau système d'exploitation ne nécessite pas le redémarrage de votre ordinateur. Vous pouvez également vous débarrasser facilement des machines virtuelles lorsque vous n'en avez pas besoin. C'est quelque chose qui ne peut pas être fait facilement avec une installation de système multi-boot typique

La virtualisation au niveau du système d'exploitation, en revanche, ne repose pas sur l'émulation du matériel. Il encapsule un environnement d'espace utilisateur (bibliothèques partagées, contraintes de ressources, volumes de système de fichiers, code, etc.) sous la forme de conteneurs qui ne peuvent pas fonctionner en dehors de l'environnement de conteneur strictement déni. Tous les conteneurs s'exécutent sur le même noyau de système d'exploitation mais ne peuvent interférer les uns avec les autres, sauf si vous les autorisez explicitement.

La virtualisation au niveau du système d'exploitation ne nécessite pas d'émulation du matériel. Néanmoins, il peut définir des contraintes spécifiques sur l'utilisation des ressources système telles que l'espace de stockage, le temps CPU, la RAM ou le réseau. Ces contraintes sont gérées uniquement par le noyau du système, de sorte que la surcharge de performances est généralement inférieure à celle de la virtualisation des machines. C'est pourquoi la virtualisation au niveau du système d'exploitation est souvent appelée virtualisation légère.

Habituellement, un conteneur ne contient que du code d'application et ses dépendances au niveau du système, principalement des bibliothèques partagées ou des binaires d'exécution comme l'interpréteur Python, mais peut être aussi volumineux que vous le souhaitez. Les images pour les conteneurs Linux sont souvent basées sur des distributions système complètes comme Debian, Ubuntu ou Fedora. Du point de vue des processus exécutés à l'intérieur d'un conteneur, cela ressemble à un environnement système complètement isolé.

En ce qui concerne l'isolation au niveau du système à des fins de développement, les deux méthodes fournissent un niveau d'isolation et de reproductibilité tout aussi suffisant. Néanmoins, en raison de sa nature plus légère, la virtualisation au niveau du système d'exploitation semble être plus favorisée par les développeurs car elle permet une utilisation moins chère, plus rapide et plus rationalisée de ces environnements, ainsi qu'un emballage et une portabilité pratiques. Ceci est particulièrement utile pour les programmeurs qui travaillent sur plusieurs projets en parallèle ou qui ont besoin de partager leurs environnements avec d'autres programmeurs.

Il existe deux principaux outils pour fournir une isolation au niveau du système des environnements de développement : 

* **Docker** pour la virtualisation au niveau du système d'exploitation
* **Vagrant** pour la virtualisation des machines

Docker et Vagrant semblent se chevaucher dans les fonctionnalités. La principale différence entre eux est la raison pour laquelle ils ont été construits. Vagrant a été construit principalement comme un outil de développement. Il vous permet d'amorcer l'ensemble de la machine virtuelle avec une seule commande, mais est rarement utilisé pour simplement compresser un tel environnement en tant qu'artefact complet qui pourrait être facilement livré à un environnement de production et exécuté tel quel. Docker, d'autre part, est conçu exactement à cette fin : préparer des conteneurs complets qui peuvent être envoyés et déployés en production en tant que package complet. S'il est bien mis en œuvre, cela peut grandement améliorer le processus de déploiement du produit.

En raison de certaines nuances d'implémentation, les environnements basés sur des conteneurs peuvent parfois se comporter différemment des environnements basés sur des machines virtuelles. Ils ne conditionnent pas non plus le noyau du système d'exploitation, donc pour le code qui est hautement spécifique au système d'exploitation, ils peuvent ne pas toujours se comporter de la même manière sur chaque hôte. De plus, si vous décidez d'utiliser des conteneurs pour le développement, mais que vous ne décidez pas de les utiliser sur des environnements de production cibles, vous perdrez certaines des garanties de cohérence qui étaient la principale raison de l'isolement de l'environnement.

Mais, si vous utilisez déjà des conteneurs dans vos environnements de production cibles, vous devez toujours reproduire les conditions de production au stade du développement en utilisant la même technique. Heureusement, Docker, qui est actuellement la solution de conteneur la plus populaire, fournit un incroyable outil de composition de docker qui rend la gestion des environnements conteneurisés locaux extrêmement facile.

Les conteneurs sont une excellente alternative à la virtualisation complète des machines. Il s'agit d'une méthode légère de virtualisation, où le noyau et le système d'exploitation permettent l'exécution de plusieurs instances isolées de l'espace utilisateur. Si votre système d'exploitation prend en charge les conteneurs de manière native, cette méthode de virtualisation nécessitera moins de temps système que la virtualisation complète de la machine.



## Environnements virtuels utilisant Docker

Les conteneurs logiciels ont obtenu leur popularité principalement grâce à Docker, qui est l'une des implémentations disponibles pour le système d'exploitation Linux. Docker vous permet de décrire une image du conteneur sous la forme d'un simple document texte appelé Dockerfile. Les images à partir de telles définitions peuvent être créées et stockées dans des référentiels d'images. Les référentiels d'images permettent à plusieurs programmeurs de réutiliser des images existantes sans avoir besoin de les créer toutes elles-mêmes. Docker prend également en charge les modifications incrémentielles, donc si de nouvelles choses sont ajoutées au conteneur, il n'a pas besoin d'être recréé à partir de zéro.

Docker est une méthode de virtualisation du système d'exploitation pour les systèmes d'exploitation Linux, il n'est donc pas pris en charge nativement par les noyaux de Windows et macOS. Néanmoins, cela ne signifie pas que vous ne pouvez pas utiliser Docker sous Windows ou macOS. Sur ces systèmes d'exploitation, Docker devient une sorte d'hybride entre la virtualisation des machines et la virtualisation au niveau du système d'exploitation. L'installation de Docker sur ces deux systèmes créera une machine virtuelle intermédiaire avec le système d'exploitation Linux qui servira d'hôte pour vos conteneurs. Le démon Docker et les utilitaires de ligne de commande s'occuperont de proxy tout trafic et images entre votre propre système d'exploitation et les conteneurs s'exécutant sur cette machine virtuelle de manière transparente.

    Vous pouvez trouver les instructions d'installation de Docker sur https://www.docker.com/get-started.

L'existence d'une machine virtuelle intermédiaire signifie que Docker sous Windows ou macOS n'est pas aussi léger que sous Linux. Néanmoins, la surcharge de performances ne devrait pas être sensiblement plus élevée que la surcharge de performances d'autres environnements de développement basés strictement sur la virtualisation des machines



## Rédiger votre premier Dockerfile

Chaque environnement basé sur Docker commence par un Dockerfile. Un Dockerfile est une description de la façon de créer une image Docker. Vous pouvez penser aux images Docker de la même manière que vous penseriez aux images de machines virtuelles. Il s'agit d'un fichier unique (composé de plusieurs couches) qui encapsule toutes les bibliothèques système, fichiers, code source et autres dépendances nécessaires à l'exécution de votre application.

Chaque couche d'une image Docker est décrite dans le Dockerfile par une seule instruction au format suivant:

    INSTRUCTION arguments

Docker prend en charge de nombreuses instructions, mais les plus élémentaires que vous devez connaître pour commencer sont les suivantes :

* **FROM  \<image-name>** : Ceci décrit l'image de base sur laquelle votre image sera basée. Ils sont souvent basés sur des distributions système Linux courantes et sont généralement livrés avec des bibliothèques et des logiciels supplémentaires installés. Le référentiel d'images Docker par défaut s'appelle Docker Hub. Il peut être consulté gratuitement et consulté sur https://hub.docker.com/.

* **COPIER \<src>... \<dst>** : cela copie les fichiers du contexte de construction local (généralement les fichiers du projet) et les ajoute au système de fichiers du conteneur

* **ADD \<src>... \<dst>** : cela fonctionne de la même manière que COPY mais décompresse automatiquement les archives et permet à \<src> d'être des URL

* **RUN \<commande>** : exécute une commande spécifiée au-dessus des couches précédentes. Après exécution, il valide les modifications apportées par cette commande au système de fichiers en tant que nouvelle couche d'image.

* **ENTRYPOINT ["\<executable>", "\<param>", ...]** : Ceci congfigure la commande par défaut à exécuter au démarrage de votre conteneur. Si aucun point d'entrée n'est spécifié dans les calques d'image, Docker utilise par défaut /bin/sh -c, qui est le shell par défaut d'une image donnée (généralement Bash mais peut également être un autre shell)

* **CMD ["\<param>", ...]** : Ceci spécie les paramètres par défaut pour les points d'entrée de l'image. Sachant que le point d'entrée par défaut de Docker est /bin/sh -c, cette instruction peut également prendre la forme de CMD ["<executable>", "<param>", ...]. Il est recommandé de définir l'exécutable cible directement dans l'instruction ENTRYPOINT et d'utiliser CMD uniquement pour les arguments par défaut.

* **WORKDIR \<dir>** : cela définit le répertoire de travail actuel pour l'une des instructions RUN, CMD, ENTRYPOINT, COPY et ADD suivantes

Pour bien illustrer la structure typique d'un Dockerfile, nous allons essayer de dockeriser une simple application Python. Imaginons que nous souhaitions créer un serveur Web d'écho HTTP qui répond avec les détails de la requête HTTP qu'il a reçue. Nous utiliserons Flask, qui est un microframework web Python très populaire.

Le code de notre application, qui serait enregistré dans un script Python, echo.py, pourrait être le suivant:




In [ ]:
from flask import Flask, request

app = Flask(__name__)


@app.route("/")
def echo():
    print(request.headers)
    return (
        f"METHOD: {request.method}\n"
        f"HEADERS:\n{request.headers}"
        f"BODY:\n{request.data.decode()}"
    )


if __name__ == "__main__":
    app.run(host="0.0.0.0")

Notre script commence par l'import de la classe Flask et de l'objet request. L'instance de la classe Flask représente notre application Web. L'objet de requête est un objet global spécial qui représente toujours le contexte de la requête HTTP actuellement traitée

echo() est une fonction dite d'affichage, qui est responsable du traitement des requêtes entrantes. @app.route('/') enregistre la fonction d'affichage echo() sous le chemin /. Cela signifie que seules les requêtes correspondant au chemin / seront envoyées à cette fonction d'affichage. À l'intérieur de notre vue, nous lisons les détails de la demande entrante (méthode, en-têtes et corps) et les renvoyons sous forme de texte. Flask inclura cette sortie de texte dans le corps de la réponse de la demande

Notre script se termine par l'appel à la méthode app.run(). Il démarre le serveur de développement local de notre application. Ce serveur de développement n'est pas destiné à être utilisé dans un environnement de production mais est suffisamment bon pour le développement et simplifie grandement notre exemple.

Si vous avez installé le package Flask, vous pouvez exécuter votre application à l'aide de la commande suivante
    
    python3 echo.py

La commande ci-dessus démarrera le serveur de développement Flask sur le port 5000. Vous pouvez soit visiter l'adresse http://localhost:5000 dans votre navigateur, soit utiliser l'utilitaire de ligne de commande.

Voici un exemple d'appel d'une requête GET à l'aide de curl

     curl localhost:5000
     METHOD: GET
     HEADERS:
     Host: localhost:5000
     User-Agent: curl/7.64.1
     Accept: */*
     
     BODY:

Comme nous avons confirmé que notre application a renvoyé les détails HTTP de la requête qu'elle a reçue, nous sommes presque prêts à la dockeriser. La structure de nos fichiers de projet pourrait être la suivante:

    .├── Dockerfile
     ├── echo.py
     └── requirements.txt

Le fichier requirements.txt ne contiendra qu'une seule entrée, flask==1.1.2, pour s'assurer que notre image utilisera toujours la même version de Flask. Avant de passer au Dockerfile, décidons comment nous voulons que notre image fonctionne. Ce que nous voulons réaliser est le suivant :

*  Cacher une certaine complexité à l'utilisateur, en particulier le fait que nous utilisons Python et Flask

* Empaqueter l'exécutable Python 3.9 avec toutes ses dépendances

* Empaqueter toutes les dépendances du projet définies dans le fichier requirements.txt


Connaissant les exigences ci-dessus, nous sommes prêts à rédiger notre premier Dockerfile. Il prendra la forme suivante:


    FROM python:3.9-slim
    WORKDIR /app/

    COPY requirements.txt .
    RUN pip install -r requirements.txt

    COPY echo.py .
    CMD ["python", "echo.py"]

FROM python:3.9-slim dénit l'image de base pour notre image de conteneur personnalisée. Python possède une collection d'images ofcielles sur Docker Hub et python:3.9-slim en fait partie. 3.9-slim est la balise de l'image comprenant Python 3.9 avec seulement un ensemble minimal de packages système nécessaires pour exécuter Python. C'est généralement un point de départ judicieux pour les images d'applications basées sur Python.

Dans la section suivante, nous apprendrons comment créer une image Docker à partir du Dockerfile ci-dessus et comment exécuter notre conteneur

## Conteneurs en cours d'exécution

Avant de pouvoir démarrer votre conteneur, vous devez d'abord créer une image définie dans le Dockerfile. Vous pouvez construire l'image en utilisant la commande suivante:

    docker build -t <name> <path>

L'argument -t \<nom> permet de nommer l'image avec un identifiant lisible. C'est totalement facultatif, mais sans cela, vous ne pourrez pas facilement référencer une image nouvellement créée. L'argument \<path> spécifie le chemin d'accès au répertoire où se trouve votre Dockerfile. Supposons que nous exécutions déjà la commande à partir de la racine du projet présenté dans la section précédente. Nous voulons également marquer notre image avec le nom echo. L'invocation de la commande docker build sera la suivante:

    docker build -t echo .

Une fois créée, vous pouvez inspecter la liste des images disponibles à l'aide de la commande docker images.

     $ docker images
     REPOSITORY      TAG       IMAGE ID         CREATED              SIZE
     echo            latest    0549d15959ef     About a minute ago   126MB
     python          3.9-slim  a90139e6bc2f     10 days ago          115MB


    La taille choquante des images de conteneur
    
    Notre image a une taille de 126 Mo car elle capture en fait l'ensemble de
    la distribution du système Linux nécessaire pour exécuter notre application
    Python. Cela peut sembler beaucoup, mais ce n'est pas vraiment de quoi
    s'inquiéter. Par souci de concision, nous avons utilisé une image de base
    simple à utiliser. Il existe d'autres images spécialement conçues pour
    minimiser cette taille, mais elles sont généralement dédiées aux
    utilisateurs de Docker plus expérimentés. De plus, grâce à la structure en
    couches des images Docker, si vous utilisez de nombreux conteneurs, les
    couches de base peuvent être mises en cache et réutilisées, de sorte qu'un
    éventuel surcoût d'espace est rarement un problème. Dans l'exemple
    précédent, la taille totale de stockage utilisée pour les deux images ne
    sera que de 126 Mo car l'image echo:latest n'ajoute que 11 Mo au-dessus de
    l'image python:3.9-slim


Une fois votre image créée et balisée, vous pouvez exécuter un conteneur à l'aide de la commande docker run. Notre conteneur est un exemple de service Web, nous devrons donc en plus dire à Docker que nous voulons publier les ports du conteneur en les liant localement.

    docker run -it --rm --publish 5000:5000 echo

Voici une explication des arguments spéciques de la commande précédente :

*  -it : Il s'agit en fait de deux options concaténées : -i et -t. Le -i (pour interactif) maintient STDIN ouvert, même si le processus conteneur est détaché, et -t (pour tty) alloue un pseudo-TTY pour le conteneur. TTY signifie téléimprimeur et sur les systèmes d'exploitation de type Linux et UNIX, représente le terminal connecté à l'entrée et à la sortie standard d'un programme. En bref, grâce à ces deux options, nous pourrons voir les journaux en direct de notre application et nous assurer que l'interruption du clavier entraînera la sortie du processus. Il se comportera simplement de la même manière que nous démarrions Python, directement à partir de la ligne de commande.

* --rm : indique à Docker de supprimer automatiquement le conteneur lorsqu'il se ferme. Sans cette option, le conteneur sera conservé afin que vous puissiez le rattacher afin de diagnostiquer son état. Par défaut, Docker ne supprime pas les conteneurs uniquement pour faciliter le débogage. Ils peuvent rapidement s'accumuler sur votre disque. Une bonne pratique consiste donc à utiliser --rm par défaut, à moins que vous n'ayez vraiment besoin de conserver le conteneur quitté pour un examen ultérieur. 

* --publish 5000:5000 : indique à Docker de publier le port 5000 du conteneur en liant le port 5000 sur l'interface de l'hôte. Vous pouvez utiliser cette option pour remapper également les ports d'application. Si vous souhaitez, par exemple, exposer l'application echo sur le port 8080 localement, vous pouvez utiliser l'argument --publish 8080:5000.


Construire et exécuter vos propres images à l'aide de la commande docker est assez simple et direct, mais peut devenir fastidieux après un certain temps. Cela nécessite l'utilisation d'appels de commandes assez longs et la mémorisation de nombreux identifiants personnalisés. Cela peut être assez gênant pour des environnements plus complexes. Dans la section suivante, nous verrons comment un workflow Docker peut être simplifié avec l'utilitaire Docker Compose.




## Mise en place d'environnements complexes

Bien que l'utilisation de base de Docker soit assez simple pour les configurations de base, elle peut être un peu écrasante une fois que vous commencez à l'utiliser dans plusieurs projets. Il est vraiment facile d'oublier les options de ligne de commande spécifiques, ou quels ports doivent être publiés sur quelles images.

Mais les choses commencent à devenir vraiment compliquées lorsque vous avez un service qui doit communiquer avec les autres. Un seul conteneur Docker ne doit contenir qu'un seul processus en cours d'exécution.

Cela signifie que vous ne devriez vraiment pas mettre d'outils de supervision de processus supplémentaires, tels que Supervisor et Circus, dans l'image du conteneur, mais plutôt configurer plusieurs conteneurs qui communiquent entre eux. Chaque service peut utiliser une image complètement différente, fournir différentes options de configuration et exposer des ports qui peuvent se chevaucher ou non. Si vous souhaitez exécuter plusieurs processus différents, chaque processus doit être un conteneur distinct.

Les grands déploiements de production de conteneurs utilisent des systèmes d'orchestration de conteneurs dédiés tels que Kubernetes, Nomad ou Docker Swarm pour suivre tous les conteneurs et leurs détails d'exécution tels que les images, les ports, les volumes, les ports, la configuration, etc. Vous pouvez utiliser l'un de ces outils localement, mais ce serait exagéré à des fins de développement.

Le meilleur outil de développement de conteneurs que vous pouvez utiliser sur votre ordinateur et qui fonctionne bien pour les scénarios simples et complexes est Docker Compose. Docker Compose est généralement distribué avec Docker, mais dans certaines distributions Linux (par exemple, Ubuntu), il peut ne pas être disponible par défaut. Dans un tel cas, il doit être installé en tant que package distinct du référentiel de packages système. Docker Compose fournit un puissant utilitaire de ligne de commande nommé docker-compose et vous permet de décrire des applications multi-conteneurs à l'aide de la syntaxe YAML.

Compose s'attend à ce que le fichier docker-compose.yml spécialement nommé se trouve dans le répertoire racine de votre projet. Un exemple d'un tel fichier pour notre projet précédent pourrait être le suivant:


    version: '3.8'

    services:
      echo-server:
      # this tell Docker Compose to build image from
      # local (.) directory
      build: .

      # this is equivalent to "-p" option of
      # the "docker build" command
      ports:
      - "5000:5000"

      # this is equivalent to "-t" option of
      # the "docker build" command
      tty: true

Si vous créez un tel fichier docker-compose.yml dans votre projet, alors tout votre environnement d'application peut être démarré et arrêté avec deux commandes simples : 

* docker-compose up : cela démarre tous les conteneurs définis dans le docker-compose. yml et imprime activement leur sortie standard

* docker-compose down : cela arrête tous les conteneurs démarrés par docker-compose dans le répertoire du projet actuel

Docker Compose construira automatiquement votre image si elle n'a pas encore été construite. C'est un excellent moyen d'encoder l'environnement de développement dans le fichier de configuration. Si vous travaillez avec d'autres programmeurs, vous pouvez fournir un fichier docker-compose.yml pour votre projet. De cette façon, la mise en place d'un environnement de développement local entièrement fonctionnel sera une question d'une seule commande docker-compose up. Le fichier docker-compose.yml doit être définitivement versionné avec le reste de votre code si vous utilisez les outils de versioning de code.

De plus, si votre application nécessite des services externes supplémentaires, vous pouvez facilement les ajouter à votre environnement Docker Compose au lieu de les installer sur votre système hôte. Considérez l'exemple suivant qui ajoute une instance d'une base de données PostgreSQL et un stockage de mémoire Redis à l'aide d'images officielles Docker Hub


    version: '3.8'

    services:
    echo-server:
      # this tell Docker Compose to build image from
      # local (.) directory
      build: .

      # this is equivalent to "-p" option of
      # the "docker build" command
      ports:
      - "5000:5000"

      # this is equivalent to "-t" option of
      # the "docker build" command
      tty: true

    database:
      image: postgres

    cache:
      image: redis

C'est aussi simple que ça. Pour assurer une meilleure reproductibilité, vous devez toujours spécifier les balises de version des images externes (comme postgres:13.1 et redis:6.0.9). De cette façon, vous vous assurerez que tous les utilisateurs de votre fichier docker-compose.yml utiliseront exactement les mêmes versions des services externes. Grâce à Docker Compose, vous pouvez utiliser plusieurs versions du même service simultanément sans aucune interférence. C'est parce que différents environnements Docker Compose sont par défaut isolés au niveau du réseau



## Recettes Docker et Docker Compose utiles pour Python

Docker et les conteneurs en général sont un sujet tellement vaste qu'il est impossible de les couvrir dans une courte section de ce guide. Grâce à Docker Compose, il est vraiment facile de commencer à travailler avec Docker sans trop savoir comment cela fonctionne en interne. Si vous êtes nouveau sur Docker, vous devrez éventuellement ralentir un peu, prendre la documentation Docker et la lire attentivement.

Voici quelques conseils et recettes rapides qui vous permettent de différer ce moment et de résoudre la plupart des problèmes courants auxquels vous devrez peut-être faire face tôt ou tard.

### Réduire la taille des conteneurs

Une préoccupation commune des nouveaux utilisateurs de Docker est la taille de leurs images de conteneur. Il est vrai que les conteneurs fournissent beaucoup d'espace supplémentaire par rapport aux packages Python simples, mais ce n'est généralement rien si nous comparons cela à la taille des images pour les machines virtuelles. Cependant, il est encore très courant d'héberger de nombreux services sur une seule machine virtuelle, mais avec une approche basée sur des conteneurs, vous devriez certainement avoir une image distincte pour chaque service. Cela signifie qu'avec de nombreux services, le surcoût peut devenir perceptible.

Si vous souhaitez limiter la taille de vos images, vous pouvez utiliser deux techniques complémentaires :

* Utilisez une image de base spécialement conçue à cet effet : Alpine Linux est un exemple de distribution Linux compacte spécialement conçue pour fournissent des images Docker très petites et légères. L'image de base fait environ 5 Mo et fournit un gestionnaire de paquets élégant qui vous permet de garder vos images compactes.

* Prenez en considération les caractéristiques du système de fichiers de superposition Docker : les images Docker sont constituées de couches où chaque couche encapsule la différence dans le système de fichiers racine entre elle-même et la couche précédente. Une fois le calque validé, la taille de l'image ne peut plus être réduite. Cela signifie que si vous avez besoin d'un package système en tant que dépendance de construction et qu'il peut être supprimé ultérieurement de l'image, au lieu d'utiliser plusieurs instructions RUN, il peut être préférable de tout faire dans une seule instruction RUN avec des commandes shell enchaînées pour éviter commits de couche excessifs

Ces deux techniques peuvent être illustrées par le Dockerfile suivant:


    FROM alpine:3.13
    WORKDIR /app/

    RUN apk add --no-cache python3

    COPY requirements.txt .
    
    RUN apk add --no-cache py3-pip && \
        pip3 install -r requirements.txt && \
        apk del py3-pip
    
    COPY echo.py .
    CMD ["python", "echo.py"]



.
    
    L'exemple ci-dessus utilise l'image de base alpine:3.12 pour illustrer la
    technique de nettoyage des dépendances inutiles avant de valider la couche.
    Malheureusement, le gestionnaire apk de la distribution Alpine ne permet
    pas de contrôler correctement quelle version de Python sera installée.
    C'est pourquoi les images de base Alpine recommandées pour les projets
    Python proviennent du référentiel officiel Python. Pour Python 3.9, ce
    serait l'image de base python:3.9-alpine

l'indicateur --no-cache d'apk (le gestionnaire de paquets d'Alpine) a deux effets. Tout d'abord, apk ignorera le cache existant des listes de packages afin qu'il installe la dernière version de package disponible officiellement dans le référentiel de packages. Deuxièmement, il ne mettra pas à jour le cache des listes de packages existants, donc la couche créée avec cette instruction sera légèrement plus petite que l'utilisation de l'indicateur --update-cache qui est autrement nécessaire pour installer le package dans sa dernière version. La différence n'est pas si grande (probablement environ 2 Mo), mais ces petits morceaux de cache peuvent s'accumuler dans des images plus grandes qui ont de nombreuses couches d'appels apk add. Les gestionnaires de paquets de différentes distributions Linux offrent généralement une manière similaire de désactiver leurs caches de liste de paquets.

La deuxième instruction RUN est un exemple de prise en compte du fonctionnement des couches d'images Docker. Sur Alpine, le package Python n'est pas fourni avec pip installé, nous devons donc l'installer nous-mêmes. Généralement, une fois que tous les packages Python requis ont été installés, pip n'est plus requis et peut être supprimé. Nous pourrions utiliser le module Ensurepip pour amorcer pip, mais nous n'aurions alors pas de moyen évident de le supprimer. Au lieu de cela, nous utilisons une instruction à longue chaîne qui s'appuie sur apk pour installer le package py3-pip et le supprimer après avoir installé les autres packages Python. Cette astuce sur Alpine 3.13 peut même nous faire gagner jusqu'à 16 Mo.

Si vous exécutez la commande Docker images, vous verrez qu'il existe une différence de taille substantielle entre les images basées sur Alpine et python:slim base images

    $ docker images
    REPOSITORY    TAG        IMAGE ID         CREATED              SIZE
    echo-alpine   latest     e7e3a2bc7b71     About a minute ago   53.7MB
    echo          latest     6b036d212e8f     40 minutes ago       126MB

L'image résultante est maintenant plus de deux fois plus petite que celle basée sur l'image python:3.9-slim. Cela est principalement dû à une distribution Alpine simplifiée qui représente environ 5 Mo au total. Sans notre astuce de suppression de pip et d'utilisation du drapeau --no-cache, la taille de l'image serait probablement d'environ 72 Mo (les caches des listes de paquets font environ 2 Mo, py3-pip environ 16 Mo). Au total cela nous a permis d'économiser près de 25% de la taille. Une telle réduction de taille ne sera pas si significative pour les applications plus volumineuses avec plus de dépendances où 18 Mo ne font pas une grande différence. Néanmoins, cette technique peut être utilisée pour d'autres dépendances au moment de la construction. Certains paquets, par exemple, nécessitent des compilateurs supplémentaires comme gcc (GNU Compiler Collection) et des fichiers d'en-tête supplémentaires au moment de l'installation. Dans une telle situation, vous pouvez utiliser le même modèle pour éviter d'avoir la collection complète du compilateur GNU dans l'image finale. Et cela peut en fait avoir un impact assez important sur la taille de l'image


## Adressage des services dans un environnement Docker Compose

Les applications complexes se composent souvent de plusieurs services qui communiquent entre eux. Compose nous permet de définir facilement de telles applications. Voici un exemple de fichier docker-compose.yml qui définit l'application comme une composition de deux services :

    version: '3.8'
    
    services:  
      echo-server:    
        build: .    
        ports:
        - "5000:5000"
        tty: true 
      
      database:    
        image: postgres
        restart: always


La configuration précédente dénit deux services :

* echo-server : il s'agit de notre conteneur de services d'application echo avec l'image construite à partir de la base de données Dockerfile locale

* tatabase : il s'agit d'un conteneur de base de données PostgreSQL provenant d'une image officielle postgres Docker


Nous supposons que le service echo-server veut communiquer avec le service de base de données sur le réseau. Afin de configurer de telles communications, nous devons connaître l'adresse IP ou le nom d'hôte du service afin qu'il puisse être utilisé comme configuration d'application. Heureusement, Docker Compose est un outil qui a été conçu exactement pour de tels scénarios, ce qui nous facilitera grandement la tâche.

Chaque fois que vous démarrez votre environnement avec la commande docker-compose up, Docker Compose créera par défaut un réseau Docker dédié et enregistrera tous les services de ce réseau en utilisant leurs noms comme noms d'hôte. Cela signifie que le service echo-server peut utiliser l'adresse database:5432 pour communiquer avec la base de données (5432 est le port PostgreSQL par défaut), et tout autre service dans cet environnement Docker Compose pourra accéder au point de terminaison HTTP de l'écho- service serveur sous l'adresse http://echo-server:80.

Même si les noms d'hôte des services dans Docker Compose sont facilement prévisibles, il n'est pas recommandé de coder en dur des adresses dans le code de votre application. La meilleure approche serait de les fournir en tant que variables d'environnement pouvant être lues par votre application au démarrage. L'exemple suivant montre comment des variables d'environnement arbitraires 

    version: '3.8'
    
    services:  
      echo-server:
          build: .    
          ports:
          - "5000:5000"
          tty: true
          environment:
              - DATABASE_HOSTNAME=database
              - DATABASE_PORT=5432
              - DATABASE_PASSWORD=password

      database:    
        image: postgres
        restart: always
        environment:
            POSTGRES_PASSWORD: password

Les lignes en **environment** fournissent des variables d'environnement qui indiquent à notre serveur d'écho quels sont le nom d'hôte et le port de la base de données. Les variables d'environnement sont le moyen le plus recommandé de fournir des paramètres de configuration pour les conteneurs

    Les conteneurs Docker sont éphémères. Cela signifie qu'une fois le conteneur
    supprimé (généralement à la sortie),
    ses modifications internes du système de fichiers sont perdues. Pour les
    bases de données, cela signifie que si vous ne voulez pas perdre de données
    dans la base de données en cours d'exécution dans le conteneur, vous devez
    monter un volume à l'intérieur d'un conteneur sous le répertoire où les
    données sont censées être stockées. Les responsables de la maintenance des
    images Docker pour les bases de données documentent généralement comment
    monter de tels volumes. Par conséquent, reportez-vous toujours à la
    documentation de l'image Docker que vous utilisez si vous souhaitez
    protéger les données de la base de données. Un exemple d'utilisation de
    volumes Docker à des fins légèrement différentes est présenté dans la
    section Ajout d'un rechargement en direct pour absolument n'importe quel code.


## Communiquer entre les environnements Docker Compose

Si vous construisez un système composé de plusieurs services et/ou applications indépendants, vous souhaiterez très probablement conserver leur code dans plusieurs référentiels de code indépendants (projets). Les fichiers docker-compose.yml de chaque application Docker Compose sont généralement conservés dans le même référentiel de code que le code de l'application. Le réseau par défaut créé par Compose pour une seule application est isolé des réseaux des autres applications. Alors, que pouvez-vous faire si vous souhaitez soudainement que vos multiples applications indépendantes communiquent entre elles ?

Heureusement, c'est une autre chose qui est extrêmement facile avec Compose. La syntaxe du fichier docker-compose.yml vous permet de définir un réseau Docker externe nommé comme réseau par défaut pour tous les services définis dans cette configuration.

Ce qui suit est un exemple de configuration qui définit un réseau externe nommé my-interservice-network

    version: '3.8'

    networks:
      default:
        external:
          name: my-interservice-network

    services:
      webqerver:
        build: .
        ports:
        - "80:80"
        tty: true
        environment:
          - DATABASE_HOSTNAME=database
          - DATABASE_PORT=5432
          - DATABASE_PASSWORD=password

      database:    
        image: postgres
        restart: always
        environment: 
             POSTGRES_PASSWORD: password

Ces réseaux externes ne sont pas gérés par Docker Compose, vous devrez donc les créer manuellement avec la commande docker network create, comme suit

    docker network create my-interservice-network

Une fois que vous avez fait cela, vous pouvez utiliser ce réseau externe dans d'autres chiers docker-compose.yml pour toutes les applications qui devraient avoir leurs services enregistrés dans le même réseau. Ce qui suit est un exemple de configuration pour d'autres applications qui pourront communiquer à la fois avec les services de base de données et de serveur Web via my-interservice-network, même s'ils ne sont pas définis dans le même fichier docker-compose.yml

    version: '3.8'

    networks:
      default:
        external:
          name: my-interservice-network

    services:
      other-service:
        build: .
        ports:
        - "80:80"
        tty: true
        environment:
          - DATABASE_HOSTNAME=database
          - DATABASE_PORT=5432
          - ECHO_SERVER_ADDRESS=http://echo-server:80


L'approche ci-dessus vous permet de démarrer deux environnements Docker Compose indépendants dans des shells séparés. Tous les services pourront communiquer entre eux via un réseau Docker partagé

## Retarder le démarrage du code jusqu'à ce que les ports de service soient ouverts

Si vous exécutez docker-compose up, tous les services seront démarrés en même temps. Vous pouvez contrôler dans une certaine mesure le démarrage du service à l'aide de la clé depend_on dans la définition du service, comme dans l'exemple suivant :

    version: '3.8'
    
    services:  
      echo-server:
          build: .
          ports:    
          - "5000:5000"
          tty: true
          depends_on:
                - database
                
        database:
          image: postgres
          environment:
              POSTGRES_PASSWORD: password


La configuration précédente s'assurera que notre serveur d'écho sera démarré après le service de base de données. Malheureusement, il ne suffit pas toujours d'assurer un bon ordre de démarrage des services dans l'environnement de développement.

Considérons une situation où echo-server devrait lire quelque chose dans la base de données immédiatement après le démarrage. Docker Compose s'assurera que les services seront démarrés dans l'ordre mais ne s'assurera pas que PostgreSQL sera prêt à accepter les connexions du serveur d'écho. C'est parce que l'initialisation de PostgreSQL peut prendre quelques secondes.

La solution pour cela est assez simple. Il existe de nombreux utilitaires de script qui vous permettent de tester si un port réseau spécifique est ouvert avant de procéder à l'exécution d'une commande. L'un de ces utilitaires s'appelle wait-for-it et est en fait écrit en Python, vous pouvez donc l'installer facilement avec pip

Vous pouvez appeler wait-for-it en utilisant la syntaxe suivante

    wait-for-it --service <service-address> -- command [...]

Le modèle d'utilisation de la commande -- [...] est un modèle courant pour les utilitaires qui encapsulent différentes exécutions de commandes où [...] représente n'importe quel ensemble d'arguments pour la commande. Le processus wait-for-it essaiera de créer une connexion TCP et lorsqu'il réussira, il exécutera la commande [...]. Par exemple, si nous souhaitons attendre la connexion localhost sur le port 2000 avant de lancer la commande python echo.py, nous exécuterons simplement

    wait-for-it --service localhost:2000 -- python echo.py

Ce qui suit est un exemple d'un fichier docker-compose.yml modifié qui remplace élégamment la commande d'image Docker par défaut et la décore avec l'appel à l'utilitaire wait-for-it pour garantir que notre serveur d'écho ne démarre que lorsqu'il serait possibilité de se connecter à la base de données

    version: '3.8'
    
    services:
      echo-server:
          build: .
          ports:
          - "5000:5000"
          tty: true    
          depends_on:
                - database    
          command:
                wait-for-it --service database:5432 --      python echo.py  
                
      database:
        image: postgres
        environment:
              POSTGRES_PASSWORD: password


wait-for-it par défaut expire après 15 secondes. Après ce délai, il démarrera le processus après la marque --, qu'il ait réussi ou non à se connecter. Vous pouvez désactiver le délai d'attente à l'aide de l'argument --timeout 0. Sans le délai d'attente, wait-for-it attendra indéfiniment

## Ajout de rechargement en direct pour absolument n'importe quel code

Lors du développement d'une nouvelle application, nous travaillons généralement avec le code de manière itérative. Nous mettons en œuvre les changements et voyons les résultats. Nous vérifions le code manuellement ou exécutons les tests. Il y a une boucle de rétroaction constante.

Avec Docker, nous devons inclure notre code dans l'image du conteneur pour le faire fonctionner. Mais exécuter docker build ou docker-compose build à chaque fois que vous apportez une modification à votre système hôte serait hautement contre-productif.


C'est pourquoi le meilleur moyen de fournir du code au conteneur tout en travaillant avec Docker au stade du développement consiste à utiliser les volumes Docker. L'idée est de lier votre répertoire de système de fichiers local au chemin du système de fichiers interne du conteneur. De cette façon, toute modification apportée aux fichiers dans le système de fichiers de l'hôte sera automatiquement répercutée à l'intérieur du conteneur. Avec Docker Compose, c'est extrêmement simple car il permet de dénir des volumes dans la configuration du service. Ce qui suit est une version modifiée de notre fichier docker-compose.yml pour le service echo qui monte le répertoire racine du projet sous le chemin /app/


    version: '3.8'

    services:
      echo-server:
        build: .
        ports:
        - "5000:5000"
        tty: true
        volumes:
          - .:/app/

Les modifications qui se produisent sur les volumes Docker montés sont activement propagées des deux côtés. De nombreux frameworks ou serveurs Python prennent en charge le rechargement à chaud actif chaque fois qu'ils remarquent que votre code a changé. Cela améliore considérablement l'expérience de développement car vous pouvez voir comment le comportement de votre application change au fur et à mesure que vous l'écrivez et sans avoir besoin de redémarrage manuel.


Tous les morceaux de code que vous écrivez ne seront probablement pas construits à l'aide d'un framework prenant en charge le rechargement actif. Heureusement, il existe un excellent package Python nommé watchdog qui vous permet de recharger n'importe quelle application observant les modifications de code. Ce paquet fournit un utilitaire watchmedo pratique qui, de la même manière que wait-for-it, peut envelopper n'importe quelle exécution de processus.

    L'utilitaire watchmedo du package watchdog nécessite des dépendances
    supplémentaires pour s'exécuter. Pour installer ce package avec des
    dépendances supplémentaires, utilisez la syntaxe d'installation pip suivante:
    pip install watchdog[watchmedo]



Ce qui suit est le format d'utilisation de base pour recharger les processus spécifiés chaque fois qu'il y a une modification d'un fichier Python dans le répertoire de travail actuel :

    watchmedo auto-restart --patterns "*.py" --recursive -- command [...]


les options --patterns "*.py" indiquent quels fichiers le processus watchmedo doit surveiller pour les changements. L'indicateur --recursive le fait traverser le répertoire de travail actuel de manière récursive afin qu'il puisse récupérer les modifications apportées même si elles sont imbriquées profondément dans l'arborescence des répertoires. Le modèle d'utilisation de la commande -- [...] est le même que la commande wait-for-it mentionnée dans Retarder le démarrage du code jusqu'à ce que les ports de service soient ouverts. Cela signifie simplement que tout ce qui suit la marque -- sera traité comme une seule commande avec des arguments (facultatifs). watchmedo lance cette commande et la redémarre chaque fois qu'il découvre un changement dans les fichiers surveillés.

Si vous installez le package watchdog dans votre image Docker, vous pourrez l'inclure élégamment dans votre docker-compose.yml comme dans l'exemple suivant :

    version: '3.8'

    services:
      echo-server:
        build: .
        ports:
        - "5000:5000"
        tty: true
        command:
          watchmedo auto-restart --patterns "*.py" --recursive --
          python echo.py
        volumes:
          - .:/app/

La configuration de Docker Compose ci-dessus redémarrera le processus à l'intérieur d'un conteneur chaque fois qu'il y aura une modification de votre code Python. Dans notre exemple, ce sera n'importe quel fichier avec l'extension .py qui réside sous le chemin /app/. Grâce au montage du répertoire source en tant que volume Docker, l'utilitaire watchmedo pourra récupérer toute modification apportée sur le système de fichiers hôte et redémarrer dès que vous enregistrez les modifications dans votre éditeur.

 Les environnements de développement avec Docker et Docker Compose sont extrêmement utiles et pratique mais ont leurs limites. La plus évidente est qu'ils ne vous permettent d'exécuter votre code que sous le système d'exploitation Linux. Même si Docker est disponible pour macOS et Windows, il repose toujours sur une machine virtuelle Linux comme couche intermédiaire, de sorte que vos conteneurs Docker fonctionneront toujours sous Linux. Si vous devez développer votre application comme si elle s'exécutait exactement sur un système spécifique différent de Linux, vous avez besoin d'une approche complètement différente de l'isolation de l'environnement. Dans la section suivante, nous allons en apprendre davantage sur un tel outil

## Environnements de développement virtuel utilisant Vagrant

Bien que Docker et Docker Compose fournissent une très bonne base pour créer des environnements de développement reproductibles et isolés, il existe des cas où une vraie machine virtuelle sera simplement un meilleur (ou le seul) choix. Un exemple d'une telle situation peut être un besoin de faire de la programmation système pour un système d'exploitation différent de Linux.


Vagrant semble actuellement être l'un des outils les plus populaires pour les développeurs pour gérer des machines virtuelles à des fins de développement local. Il fournit un moyen simple et pratique de décrire les environnements de développement avec toutes les dépendances système d'une manière directement liée au code source de votre projet. Il est disponible pour Windows, macOS et quelques distributions Linux populaires (voir https://www.vagrantup.com)

Il n'a pas de dépendances supplémentaires. Vagrant crée de nouveaux environnements de développement sous la forme de machines virtuelles ou de conteneurs. L'implémentation exacte dépend d'un choix de fournisseurs de virtualisation. VirtualBox est le fournisseur par défaut et il est fourni avec le programme d'installation de Vagrant, mais des fournisseurs supplémentaires sont également disponibles. Les choix les plus notables sont VMware, Docker, Linux Containers (LXC) et Hyper-V.

La configuration la plus importante est fournie à Vagrant dans un seul fichier nommé Vagrantfile. Il doit être indépendant pour chaque projet. Voici les éléments les plus importants qu'il fournit :

* Choix du fournisseur de virtualisation
* Un boîtier, qui est utilisé comme image de machine virtuelle
* Choix de la méthode de provisionnement
* Stockage partagé entre la machine virtuelle et l'hôte de la machine virtuelle
* Ports qui doivent être transférés entre la machine virtuelle et son hôte

Le langage de syntaxe pour un Vagrantfile est Ruby. L'exemple de fichier de configuration fournit un bon modèle pour démarrer le projet et dispose d'une excellente documentation, la connaissance de ce langage n'est donc pas requise. La configuration du modèle peut être créée à l'aide d'une seule commande

    vagrant init

Cela créera un nouveau fichier nommé Vagrantfile dans le répertoire de travail actuel. Le meilleur endroit pour stocker ce chier est généralement la racine des sources du projet associé. Ce fichier est déjà une configuration valide qui créera une nouvelle machine virtuelle en utilisant le fournisseur VirtualBox et l'image de la boîte basée sur une distribution Linux Ubuntu. Le contenu par défaut de Vagrantfile créé avec la commande vagrant init contient de nombreux commentaires qui vous guideront tout au long du processus de configuration.

Ce qui suit est un exemple minimal d'un Vagrantfile pour l'environnement de développement Python 3.9 basé sur le système d'exploitation Ubuntu, avec quelques valeurs par défaut sensées qui, entre autres, activent le transfert du port 80 au cas où vous voudriez faire du développement Web avec Python


  Vagrant.configure("2") do |config|
    # Every Vagrant development environment requires a box.
    # You can search for boxes at https://vagrantcloud.com/search.
    # Here we use Trusty version of Ubuntu system for x64 architecture.
    config.vm.box = "ubuntu/trusty64"

    # Create a forwarded port mapping which allows access to a specific
    # port within the machine from a port on the host machine and only
    # allow access via 127.0.0.1 to disable public access
    config.vm.network "forwarded_port", guest: 80, host: 8080, host_ip: "127.0.0.1"

    config.vm.provider "virtualbox" do |vb|
      vb.gui = false
      # Customize the amount of memory on the VM:
      vb.memory = "1024"
    end

    # Enable provisioning with a shell script.
    config.vm.provision "shell", inline: <<-SHELL
      apt-get update
      apt-get install python3.9 -y
    SHELL
  end


Dans l'exemple précédent, nous avons défini le provisionnement supplémentaire des packages système avec un simple script shell à l'intérieur de la section config.vm.provision. L'image de machine virtuelle par défaut fournie par la "boîte" ubuntu/bionic64 n'inclut pas la version Python 3.9, nous devons donc l'installer à l'aide du gestionnaire de packages apt-get

Lorsque vous sentez que le Vagrantfile est prêt, vous pouvez exécuter votre machine virtuelle à l'aide de la commande suivante :

    vagrant up

Le démarrage initial peut prendre quelques minutes, car l'image réelle de la boîte doit être téléchargée à partir du Web. Certains processus d'initialisation peuvent également prendre un certain temps à chaque démarrage de la machine virtuelle existante, et la durée dépend du choix du fournisseur, de l'image et des performances de votre système. Habituellement, une fois l'image téléchargée, cela ne prend que quelques secondes. Lorsque l'environnement Vagrant est opérationnel, vous pouvez vous y connecter via SSH en utilisant le raccourci shell suivant :

    vagrant ssh

Cela peut être fait n'importe où dans l'arborescence des sources du projet sous l'emplacement du fichier Vagrant. Pour la commodité des développeurs, Vagrant parcourra tous les répertoires au-dessus du répertoire de travail actuel de l'utilisateur dans l'arborescence du système de fichiers, recherchant le fichier de configuration et le faisant correspondre avec l'instance de machine virtuelle associée. Ensuite, il établit la connexion shell sécurisée, de sorte que l'environnement de développement peut être utilisé comme une machine distante ordinaire. La seule différence est que toute l'arborescence des sources du projet (racine définie comme l'emplacement du fichier Vagrant) est disponible sur le système de fichiers de la machine virtuelle sous /vagrant/. Ce répertoire est automatiquement synchronisé avec votre système de fichiers hôte, vous pouvez donc normalement utiliser un IDE ou un éditeur de code de votre choix sur l'hôte et traiter simplement la session SSH sur votre machine virtuelle Vagrant comme une session shell locale normale.

## Outils de productivité populaires

Presque tous les packages Python open source publiés sur PyPI sont une sorte de booster de productivité - ils fournissent des solutions prêtes à l'emploi à certains problèmes. De cette façon, nous n'avons pas à réinventer la roue tout le temps. Certains pourraient également dire que Python lui-même est une question de productivité. Presque tout dans cette langue et la communauté qui l'entoure semble être conçu pour rendre le développement de logiciels aussi productif que possible.

Cela crée une boucle de rétroaction positive. Comme écrire du code avec Python est amusant et facile, de nombreux programmeurs utilisent leur temps libre pour créer des outils qui le rendent encore plus facile et amusant. Et ce fait sera utilisé ici comme base pour une définition très subjective et non scientifique d'un outil de productivité - un logiciel qui rend le développement plus facile et plus amusant.

Par nature, les outils de productivité se concentrent principalement sur certains éléments du processus de développement, tels que les tests, le débogage et la gestion des packages, et ne sont pas des éléments essentiels des produits qu'ils aident à créer. Dans certains cas, ils peuvent même ne pas être référencés nulle part dans la base de code du projet, bien qu'ils soient utilisés quotidiennement.

Nous avons déjà évoqué les outils autour de la gestion des packages et de l'isolation des environnements virtuels. Ce sont sans aucun doute des outils de productivité car leur objectif est de simplifier et de faciliter les processus fastidieux de mise en place de votre environnement de travail local. Plus loin dans ce guide, nous discuterons d'autres outils de productivité qui aident à résoudre des problèmes spécifiques, tels que le proling et les tests. Cette section est dédiée à d'autres outils qui valent vraiment la peine d'être mentionnés.